# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [97]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [98]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palembang,-2.9167,104.7458,26.05,94,40,1.03,ID,1715101404
1,1,auki,-8.7676,160.7034,25.55,91,93,2.40,SB,1715101404
2,2,albany,42.6001,-73.9662,23.84,39,0,0.89,US,1715101404
3,3,yellowknife,62.4560,-114.3525,11.34,69,75,5.66,CA,1715101184
4,4,ribeira grande,38.5167,-28.7000,20.22,88,40,7.15,PT,1715101404


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [99]:
# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [100]:
# Narrow down cities that fit criteria and drop any results with null values
#max temp higher than 23 and lower than 24 (to get a smaller sample)
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] < 24) & (city_data_df["Max Temp"] > 23)]

# Drop any rows with null values
ideal_city_data_df =  ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,albany,42.6001,-73.9662,23.84,39,0,0.89,US,1715101404
29,29,tazacorte,28.6290,-17.9293,23.97,80,20,3.60,ES,1715101407
37,37,singkil,-7.9855,111.4161,23.83,94,25,2.10,ID,1715101408
38,38,rio grande,-32.0350,-52.0986,23.06,82,99,3.53,BR,1715101158
102,102,hamilton,39.1834,-84.5333,23.93,86,40,2.06,US,1715101391
120,120,beaufort west,-32.3567,22.5830,23.53,23,26,4.77,ZA,1715101417
141,141,shache,38.4167,77.2406,23.69,19,0,3.28,CN,1715101420
172,172,tura,25.5198,90.2201,23.78,67,40,1.77,IN,1715101424
181,181,kapa'a,22.0752,-159.3190,23.39,80,75,5.14,US,1715101425
191,191,loikaw,19.6742,97.2094,23.03,67,20,1.22,MM,1715101426


### Step 3: Create a new DataFrame called `hotel_df`.

In [101]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
#hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,39,
29,tazacorte,ES,28.6290,-17.9293,80,
37,singkil,ID,-7.9855,111.4161,94,
38,rio grande,BR,-32.0350,-52.0986,82,
102,hamilton,US,39.1834,-84.5333,86,
120,beaufort west,ZA,-32.3567,22.5830,23,
141,shache,CN,38.4167,77.2406,19,
172,tura,IN,25.5198,90.2201,67,
181,kapa'a,US,22.0752,-159.3190,80,
191,loikaw,MM,19.6742,97.2094,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [108]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "format": "json",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
singkil - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
hamilton - nearest hotel: North Vista Manor
beaufort west - nearest hotel: Matoppo Inn
shache - nearest hotel: 莎車新盛賓舘
tura - nearest hotel: Hotel Polo Orchid
kapa'a - nearest hotel: Pono Kai Resort
loikaw - nearest hotel: Nawaday Hotel
tucumcari - nearest hotel: Pow Wow Inn
ban nam yuen - nearest hotel: No hotel found
bahia de kino - nearest hotel: Hotel Hacienda
broome - nearest hotel: No hotel found
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
kununurra - nearest hotel: No hotel found
richards bay - nearest hotel: SUN1 Richards Bay
virac - nearest hotel: Marem Pension House
lautoka - nearest hotel: Cathay Hotel
lata - nearest hotel: Residencial Botânico
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
lupane - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pen

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,39,No hotel found
29,tazacorte,ES,28.6290,-17.9293,80,App Leyma
37,singkil,ID,-7.9855,111.4161,94,No hotel found
38,rio grande,BR,-32.0350,-52.0986,82,Hotel Vila Moura Executivo
102,hamilton,US,39.1834,-84.5333,86,North Vista Manor
120,beaufort west,ZA,-32.3567,22.5830,23,Matoppo Inn
141,shache,CN,38.4167,77.2406,19,莎車新盛賓舘
172,tura,IN,25.5198,90.2201,67,Hotel Polo Orchid
181,kapa'a,US,22.0752,-159.3190,80,Pono Kai Resort
191,loikaw,MM,19.6742,97.2094,67,Nawaday Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [114]:
# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)